# Importing libraries

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import skimage
from skimage import transform
from tensorflow import keras
from skimage import color

# Function for calcualting the accuracy of the classification

In [0]:
def acc(output,label):
    test=((output[:,0]-label)==0)*1.0
    return sum(test)/len(test)*100

# Sigmoid activation function

In [0]:
def sigmoid(x):
    return(1/(1+np.exp(-0.0001*x)))

# ReLU activation function

In [0]:
def relu(x):
    return np.where(x > 0, x, 0.1 * x)

In [0]:
def drelu(x):
    return np.where(x > 0, 1, 0.1)

# Network inputs (MNIST)

Loading data (Digits)

In [0]:
ix=28*28 # Number of features
epochs=20
digits=[0,1] # Digits to classify

# Loading data
mnist=keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


# Training

indices=np.append(np.where(train_labels==digits[0]),np.where(train_labels==digits[1]))
train=train_images[indices]
train=train.reshape(len(train),28*28)
# Additional column is added to the training set instead of calculating biases
train=np.concatenate((train,np.ones([len(train),1])),axis=1)
train_lbl=train_labels[indices]
train_lbl=(train_lbl==digits[1])*1.0

# Test

indices=np.append(np.where(test_labels==digits[0]),np.where(test_labels==digits[1]))
test=test_images[indices]
test=test.reshape(len(test),28*28)
test=np.concatenate((test,np.ones([len(test),1])),axis=1)
test_lbl=test_labels[indices]
test_lbl=(test_lbl==digits[1])*1.0

# ADALINE-mse

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]
print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=sigmoid(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*logits*(1-logits) # new
        w=w+diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 58.91 accuracy: 99.80
Epoch: 1, error: 30.78 accuracy: 99.85
Epoch: 2, error: 29.62 accuracy: 99.91
Epoch: 3, error: 14.16 accuracy: 99.83
Epoch: 4, error: 25.91 accuracy: 99.91
Epoch: 5, error: 23.86 accuracy: 99.84
Epoch: 6, error: 33.49 accuracy: 99.83
Epoch: 7, error: 14.74 accuracy: 99.90
Epoch: 8, error: 21.30 accuracy: 99.79
Epoch: 9, error: 16.55 accuracy: 99.91
Epoch: 10, error: 11.90 accuracy: 99.94
Epoch: 11, error: 15.94 accuracy: 99.81
Epoch: 12, error: 14.02 accuracy: 99.92
Epoch: 13, error: 10.14 accuracy: 99.92
Epoch: 14, error: 12.72 accuracy: 99.91
Epoch: 15, error: 16.56 accuracy: 99.87
Epoch: 16, error: 15.28 accuracy: 99.88
Epoch: 17, error: 20.59 accuracy: 99.91
Epoch: 18, error: 10.26 accuracy: 99.94
Epoch: 19, error: 12.69 accuracy: 99.90

Test:

Test accuracy: 99.95


# ADALINE-ReLU

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]

# Normalize
train /= 255
test /= 255

print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=relu(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*drelu(logits) # new
        w=w+0.005*diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 6625.79 accuracy: 96.95
Epoch: 1, error: 2872.40 accuracy: 96.53
Epoch: 2, error: 2240.41 accuracy: 95.41
Epoch: 3, error: 1899.97 accuracy: 96.50
Epoch: 4, error: 1689.66 accuracy: 97.92
Epoch: 5, error: 1532.12 accuracy: 99.11
Epoch: 6, error: 1420.56 accuracy: 98.15
Epoch: 7, error: 1322.88 accuracy: 97.78
Epoch: 8, error: 1257.95 accuracy: 96.13
Epoch: 9, error: 1186.80 accuracy: 98.72
Epoch: 10, error: 1128.50 accuracy: 98.23
Epoch: 11, error: 1080.26 accuracy: 99.21
Epoch: 12, error: 1053.83 accuracy: 95.76
Epoch: 13, error: 1014.43 accuracy: 97.96
Epoch: 14, error: 983.52 accuracy: 96.83
Epoch: 15, error: 959.73 accuracy: 97.01
Epoch: 16, error: 935.51 accuracy: 97.09
Epoch: 17, error: 916.24 accuracy: 98.56
Epoch: 18, error: 909.04 accuracy: 97.74
Epoch: 19, error: 887.59 accuracy: 97.41

Test:

Test accuracy: 98.16


# Network inputs (Cifar10)

Loading data Cifar10

In [2]:
ix=32*32 # Number of features
epochs=40
digits=[0,1] # Digits to classify

# Loading data
mnist=keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_labels=train_labels.reshape(len(train_labels))
test_labels=test_labels.reshape(len(test_labels))

# Training

indices=np.append(np.where(train_labels==digits[0]),np.where(train_labels==digits[1]))
train=train_images[indices]
train=skimage.color.rgb2gray(train)
train=train.reshape(len(train),ix)
train=np.concatenate((train,np.ones([len(train),1])),axis=1)
train_lbl=train_labels[indices]
train_lbl=(train_lbl==digits[1])*1.0

# Test

indices=np.append(np.where(test_labels==digits[0]),np.where(test_labels==digits[1]))
test=test_images[indices]
test=skimage.color.rgb2gray(test)
test=test.reshape(len(test),ix)
test=np.concatenate((test,np.ones([len(test),1])),axis=1)
test_lbl=test_labels[indices]
test_lbl=(test_lbl==digits[1])*1.0

Exception: ignored

# ADALINE-mse

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]
print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=sigmoid(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*logits*(1-logits) # new
        w=w+diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 4880.15 accuracy: 63.12
Epoch: 1, error: 4732.61 accuracy: 66.61
Epoch: 2, error: 4609.37 accuracy: 72.42
Epoch: 3, error: 4507.05 accuracy: 72.95
Epoch: 4, error: 4420.66 accuracy: 73.82
Epoch: 5, error: 4346.45 accuracy: 73.91
Epoch: 6, error: 4280.42 accuracy: 74.13
Epoch: 7, error: 4224.96 accuracy: 74.43
Epoch: 8, error: 4176.29 accuracy: 74.09
Epoch: 9, error: 4131.30 accuracy: 74.65
Epoch: 10, error: 4093.67 accuracy: 74.82
Epoch: 11, error: 4055.74 accuracy: 74.81
Epoch: 12, error: 4023.30 accuracy: 74.70
Epoch: 13, error: 3991.77 accuracy: 75.00
Epoch: 14, error: 3966.11 accuracy: 75.29
Epoch: 15, error: 3940.99 accuracy: 75.25
Epoch: 16, error: 3916.87 accuracy: 75.31
Epoch: 17, error: 3893.32 accuracy: 75.56
Epoch: 18, error: 3874.53 accuracy: 75.57
Epoch: 19, error: 3852.69 accuracy: 75.27
Epoch: 20, error: 3839.18 accuracy: 75.75
Epoch: 21, error: 3820.85 accuracy: 75.84
Epoch: 22, error: 3805.24 accuracy: 75.88
Epoch: 23, error: 3788.18 accura

# ADALINE-ReLU

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]

# Normalization (scaling)
train_mean = train[:, :1024].mean(axis=0)
train_std = train[:, :1024].std(axis=0)
train[:, :1024] = (train[:, :1024] - train_mean) / train_std
test[:, :1024] = (test[:, :1024] - train_mean) / train_std

print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=relu(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*drelu(logits) # new
        w=w+0.001*diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 13482.92 accuracy: 54.01
Epoch: 1, error: 8705.00 accuracy: 51.83
Epoch: 2, error: 8148.26 accuracy: 53.81
Epoch: 3, error: 7667.49 accuracy: 56.41
Epoch: 4, error: 7207.29 accuracy: 59.60
Epoch: 5, error: 6919.21 accuracy: 59.57
Epoch: 6, error: 6599.89 accuracy: 59.88
Epoch: 7, error: 6326.83 accuracy: 61.33
Epoch: 8, error: 6106.21 accuracy: 61.76
Epoch: 9, error: 5889.89 accuracy: 67.23
Epoch: 10, error: 5784.08 accuracy: 62.19
Epoch: 11, error: 5633.38 accuracy: 65.55
Epoch: 12, error: 5470.59 accuracy: 58.86
Epoch: 13, error: 5351.66 accuracy: 66.57
Epoch: 14, error: 5251.92 accuracy: 62.86
Epoch: 15, error: 5216.68 accuracy: 62.07
Epoch: 16, error: 5030.75 accuracy: 66.76
Epoch: 17, error: 4976.77 accuracy: 68.73
Epoch: 18, error: 4905.96 accuracy: 65.22
Epoch: 19, error: 4810.09 accuracy: 71.72
Epoch: 20, error: 4785.18 accuracy: 69.44
Epoch: 21, error: 4740.01 accuracy: 68.02
Epoch: 22, error: 4698.62 accuracy: 62.96
Epoch: 23, error: 4651.31 accur